In [1]:
import os
import sys
import numpy as np

In [2]:
import torch
from torch.autograd import Variable

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
import tensorflow as tf
from tensorflow import keras

2024-01-31 16:10:09.682421: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 16:10:09.702292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 16:10:09.702311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 16:10:09.702818: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 16:10:09.706135: I tensorflow/core/platform/cpu_feature_guar

In [16]:
import random
seed = 42;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [9]:
sys.path.append("../../")

In [10]:
from th.resources import models
from th.resources import calculator as calc

In [11]:
src_pt_model_path = "../retrained_models_after_pruned/retrained_model_20240124123209_acc_95.45455169677734_795th_epoch.pt"
random_weigths_pruned_keras_model_path = "../keras_models/pruned_model_copy_weights_from_retrained_model_20240124123209_acc_95.45455169677734_v3.keras"

In [12]:
state = torch.load(src_pt_model_path,map_location='cpu')
torch_model_weights = state['weight']
config = state['config']
test_torch_model = models.GetACDNetModel(input_len=30225, nclass=2, sr=20000, channel_config=config)
test_torch_model.load_state_dict(torch_model_weights)

<All keys matched successfully>

In [13]:
random_weigths_pruned_keras_model = keras.models.load_model(random_weigths_pruned_keras_model_path)

2024-01-31 16:10:43.757596: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-31 16:10:43.757613: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: aiw
2024-01-31 16:10:43.757615: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: aiw
2024-01-31 16:10:43.757642: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 545.29.6
2024-01-31 16:10:43.757653: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 545.29.6
2024-01-31 16:10:43.757655: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 545.29.6


In [21]:
input_torch_tensor = torch.randn(1,1,1,30225)
print(input_torch_tensor)
input_np = input_torch_tensor.numpy()
input_tf_var = tf.convert_to_tensor(np.transpose(input_np,(1,2,3,0)))
print(input_tf_var.shape)

tensor([[[[ 0.5255, -1.0540,  1.0037,  ...,  0.2200, -0.2230, -1.2847]]]])
(1, 1, 30225, 1)


In [22]:
random_weigths_pruned_keras_model.predict(input_tf_var)

1/1 [==============================] - 0s 11ms/step


array([[0.38819686, 0.6118032 ]], dtype=float32)

In [23]:
print(test_torch_model(input_torch_tensor))
print(type(test_torch_model))

tensor([[0.6598, 0.3402]], grad_fn=<SoftmaxBackward0>)
<class 'th.resources.models.ACDNetV2'>


In [20]:
random_weigths_pruned_keras_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 30225, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 1, 15109, 5)       45        
                                                                 
 batch_normalization_12 (Ba  (None, 1, 15109, 5)       20        
 tchNormalization)                                               
                                                                 
 re_lu_12 (ReLU)             (None, 1, 15109, 5)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 7553, 32)       800       
                                                                 
 batch_normalization_13 (Ba  (None, 1, 7553, 32)       128       
 tchNormalization)                                         

In [19]:
calc.summary(test_torch_model, (1,1,30225))

+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 30225)     (5, 1, 15109)         45      679,905
  BatchNorm2d-2     (5, 1, 15109)     (5, 1, 15109)         10            0
         ReLu-3     (5, 1, 15109)     (5, 1, 15109)          0       75,545
       Conv2d-4     (5, 1, 15109)     (32, 1, 7553)        800    6,042,400
  BatchNorm2d-5     (32, 1, 7553)     (32, 1, 7553)         64            0
         ReLu-6     (32, 1, 7553)     (32, 1, 7553)          0      241,696
    MaxPool2d-7     (32, 1, 7553)      (32, 1, 151)          0      241,600
      Permute-8      (32, 1, 151)      (1, 32, 151)          0            0
       Conv2d-9      (1, 32, 151)      (9, 32, 151)         81      391,392
 Ba

## pytorch conv2d NCHW || tensorflow conv2d NHWC

In [20]:
t2t_conv_pos_mov = (2,3,1,0)
# t2t_conv_pos_mov = (1,2,3,0)

In [21]:
# set the conv2d_12 weights
tmp_w = torch.permute(torch_model_weights['sfeb.0.weight'],t2t_conv_pos_mov)
random_weigths_pruned_keras_model.get_layer('conv2d_12').set_weights([tmp_w.cpu()])

In [22]:
#set the batch_normalization_12
tmp_w1 = torch_model_weights['sfeb.1.weight'].cpu()
tmp_w2 = torch_model_weights['sfeb.1.bias'].cpu()
tmp_w3 = torch_model_weights['sfeb.1.running_mean'].cpu()
tmp_w4 = torch_model_weights['sfeb.1.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_12').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [23]:
#set the conv2d_13 weights
print(torch_model_weights['sfeb.3.weight'].shape)
tmp_w = torch.permute(torch_model_weights['sfeb.3.weight'],(2,3,1,0)) 
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_13').set_weights([tmp_w.cpu()])

torch.Size([32, 5, 1, 5])
torch.Size([1, 5, 5, 32])


In [24]:
#set the batch_normalization_13
tmp_w1 = torch_model_weights['sfeb.4.weight'].cpu()
tmp_w2 = torch_model_weights['sfeb.4.bias'].cpu()
tmp_w3 = torch_model_weights['sfeb.4.running_mean'].cpu()
tmp_w4 = torch_model_weights['sfeb.4.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_13').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [25]:
#set the conv2d_14 weights
# Layer conv2d_14 weight shape (3, 3, 1, 9) is not compatible with provided weight shape torch.Size([9, 1, 3, 3]).
tmp_w = torch_model_weights['tfeb.0.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_14').set_weights([tmp_w.cpu()])

torch.Size([9, 1, 3, 3])
torch.Size([3, 3, 1, 9])


In [26]:
#set the batch_normalization_14
tmp_w1 = torch_model_weights['tfeb.1.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.1.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.1.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.1.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_14').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [27]:
#set the conv2d_15 weights
tmp_w = torch_model_weights['tfeb.4.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_15').set_weights([tmp_w.cpu()])

torch.Size([16, 9, 3, 3])
torch.Size([3, 3, 9, 16])


In [28]:
#set the batch_normalization_15
tmp_w1 = torch_model_weights['tfeb.5.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.5.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.5.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.5.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_15').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [29]:
#set the conv2d_16 weights
tmp_w = torch_model_weights['tfeb.7.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_16').set_weights([tmp_w.cpu()])

torch.Size([23, 16, 3, 3])
torch.Size([3, 3, 16, 23])


In [30]:
#set the batch_normalization_16
tmp_w1 = torch_model_weights['tfeb.8.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.8.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.8.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.8.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_16').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [31]:
#set the conv2d_17 weights
tmp_w = torch_model_weights['tfeb.11.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_17').set_weights([tmp_w.cpu()])

torch.Size([33, 23, 3, 3])
torch.Size([3, 3, 23, 33])


In [32]:
#set the batch_normalization_17
tmp_w1 = torch_model_weights['tfeb.12.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.12.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.12.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.12.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_17').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [33]:
#set the conv2d_18 weights
tmp_w = torch_model_weights['tfeb.14.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_18').set_weights([tmp_w.cpu()])

torch.Size([29, 33, 3, 3])
torch.Size([3, 3, 33, 29])


In [34]:
#set the batch_normalization_18
tmp_w1 = torch_model_weights['tfeb.15.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.15.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.15.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.15.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_18').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [35]:
#set the conv2d_19 weights
tmp_w = torch_model_weights['tfeb.18.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_19').set_weights([tmp_w.cpu()])

torch.Size([56, 29, 3, 3])
torch.Size([3, 3, 29, 56])


In [36]:
#set the batch_normalization_19
tmp_w1 = torch_model_weights['tfeb.19.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.19.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.19.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.19.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_19').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [37]:
#set the conv2d_20 weights
tmp_w = torch_model_weights['tfeb.21.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_20').set_weights([tmp_w.cpu()])

torch.Size([47, 56, 3, 3])
torch.Size([3, 3, 56, 47])


In [38]:
#set the batch_normalization_20
tmp_w1 = torch_model_weights['tfeb.22.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.22.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.22.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.22.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_20').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [39]:
#set the conv2d_21 weights
tmp_w = torch_model_weights['tfeb.25.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_21').set_weights([tmp_w.cpu()])

torch.Size([65, 47, 3, 3])
torch.Size([3, 3, 47, 65])


In [40]:
#set the batch_normalization_21
tmp_w1 = torch_model_weights['tfeb.26.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.26.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.26.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.26.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_21').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [41]:
#set the conv2d_22 weights
tmp_w = torch_model_weights['tfeb.28.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_22').set_weights([tmp_w.cpu()])

torch.Size([90, 65, 3, 3])
torch.Size([3, 3, 65, 90])


In [42]:
#set the batch_normalization_22
tmp_w1 = torch_model_weights['tfeb.29.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.29.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.29.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.29.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_22').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [43]:
#set the conv2d_23 weights
tmp_w = torch_model_weights['tfeb.33.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_23').set_weights([tmp_w.cpu()])

torch.Size([2, 90, 1, 1])
torch.Size([1, 1, 90, 2])


In [44]:
#set the batch_normalization_23
tmp_w1 = torch_model_weights['tfeb.34.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.34.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.34.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.34.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_23').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [48]:
#set the dense_1 weights
dense_w = torch_model_weights['tfeb.38.weight']
dense_b = torch_model_weights['tfeb.38.bias'].cpu()
print(dense_w.shape)
print(dense_w)
print(dense_b)
dense_b[0] = -0.1046
dense_b[1] = -0.0100
# dense_b[0], dense_b[1] = dense_b[1], dense_b[0] 
print(dense_b.shape)
dense_w = torch.transpose(dense_w,0,1).cpu()
print(dense_w.shape)
print(dense_w)
print(dense_b)

random_weigths_pruned_keras_model.get_layer('dense_1').set_weights([dense_w,dense_b])

torch.Size([2, 2])
tensor([[ 1.3877, -1.2969],
        [-1.3315,  1.4445]])
tensor([-0.1046, -0.0100])
torch.Size([2])
torch.Size([2, 2])
tensor([[ 1.3877, -1.3315],
        [-1.2969,  1.4445]])
tensor([-0.1046, -0.0100])


In [49]:
random_weigths_pruned_keras_model.save("../keras_models/pruned_model_copy_weights_from_retrained_model_20240124123209_acc_95.45455169677734_v3.h5")
random_weigths_pruned_keras_model.save("../keras_models/pruned_model_copy_weights_from_retrained_model_20240124123209_acc_95.45455169677734_v3.keras")

/home/ai/miniconda3/envs/acdnetenv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# random_weigths_pruned_keras_model.get_layer('conv2d_13').weights

In [ ]:
# torch.permute(torch_model_weights['sfeb.3.weight'],(2,1,3,0))

In [ ]:
# torch_model_weights.keys()

In [ ]:
# calc.summary(torch_model, (1,1,30225));